# ID2222 Data Mining, Homework 3
# **Mining Data Streams**

Brando Chiminelli, Tommaso Praturlon

November 28th, 2022

## Goal
The goal of this notebook is to 

## How to run

In order to run this notebook you need to 

## Import libraries and read the dataset
In the following we import the few libraries needed for the project and we read the dataset.

We decided to read the first 2000 baskets from the dataset in order to reduce weight on memory. Our assumption is that items are uniformly distributed across the dataset, thus allowing us to have a good insight only from the given sample.

In [20]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
from collections import Counter
from itertools import combinations
import statistics
import math

PATH_TO_DATA = "../data/facebook_combined.txt"
df_graphs = pd.read_csv(PATH_TO_DATA, header=None)
print("Data read successfully!")

# Reduce dataset size for computation overload (temporary)
#df_graphs = df_graphs.iloc[0:25000]
print(df_graphs.head())
print("Number of rows: ", len(df_graphs))

Data read successfully!
     0
0  0 1
1  0 2
2  0 3
3  0 4
4  0 5
Number of rows:  88234


In [21]:
# DATA WRANGLING
# 1. remove text rows from dataset
# 2. create a dataset of integers
data = []
for i in range(len(df_graphs)):
    s = [int(x) for x in str(df_graphs.iloc[i][0]).split(' ')]
    #s = str(df_g.iloc[i][0]).split('\t')
    data.append(s)
#print(data)

In [22]:
# DATA WRANGLING
# 1. remove text rows from dataset
# 2. create a dataset of integers
#df_g = df_graphs.drop([0, 1, 2, 3, 4])
#data = []
#for i in range(len(df_g)):
#    s = [int(x) for x in str(df_g.iloc[i][0]).split('\t')]
    #s = str(df_g.iloc[i][0]).split('\t')
#    data.append(s)
#print(data)

In [23]:
####################################
# TRIEST-BASE CLASS IMPLEMENTATION #
####################################

class TriestBase:
    '''
    Implementation of the Trièst-base algorithm
    - function SampleEdge
    - function UpdateCounter
    - function FlipBiasedCoin
    '''
    def __init__(self, M = 100):
        self.M = M
        self.S = []
        self.tau = 0
        self.tau_loc = {}
        self.t = 0
    
    def flipBiasedCoin(self, t):
        '''
        Flip a biased coin with probability M/t of falling head.
        '''
        # 1: head, 0: tail
        result = np.random.choice([1, 0], p=[self.M/t, (1-self.M/t)])
        if result:
            return True
        else:
            return False
    
    def sampleEdge(self, t):
        '''
        Receives the edge (u,v) as [u, v] and time t at which
        the stream element is received. t is a count integer.
        Returns a boolean
        '''
        if (t <= self.M):
            return True
        elif self.flipBiasedCoin(t):
            # select random edge from S
            random_edge = random.choice(self.S)
            # Delete random_edge from S
            self.S.remove(random_edge)
            # Update counters
            self.updateCounters('delete', random_edge)
            return True
        else:
            return False
            
    def updateCounters(self, operation, edge):
        '''
        Receives the operation insertion or deletion
        and the edge.
        tau is the global counter
        e.g S = [[5,10], [10, 3], [12, 5], [12, 3]]
        edge = (5, 12)
        N_5 = (10, 12)
        N_12 = (5, 3)
        N_5_12 = (5, 12)
        '''
        # Define shared-neighborhood
        shared_neigh = set()
        neigh_u = set() # all neighbors of edge[0]
        neigh_v = set() # all nneighbors of edge[1]
        #print("S from updateCounters():\n", S)
        for elem in self.S:
            # check that for the v in V_t (u,v) belongs to S
            # create neigh_u
            if edge[0] == elem[0]: # found u in position 0
                neigh_u.add(edge[1]) # add the other element
            if edge[0] == elem[1]: # found u in position 1
                neigh_u.add(edge[0]) # add the other element
            # create neigh_v
            if edge[1] == elem[0]: # found v in position 0
                neigh_v.add(edge[1]) 
            if edge[1] == elem[1]: # found v in position 1
                neigh_v.add(edge[0])
        # shared neighbourhood is the intersection between the sets
        shared_neigh = neigh_u.intersection(neigh_v)
        shared_neigh = set.intersection(neigh_u, neigh_v)

        for c in shared_neigh:    
            if operation == 'insert':
                # Insert
                self.tau += 1
                self.tau_loc[c] = self.tau_loc.get(c, 0) + 1
                self.tau_loc[edge[0]] = self.tau_loc.get(edge[0], 0) + 1
                self.tau_loc[edge[1]] = self.tau_loc.get(edge[1], 0) + 1
            
            else:
                # Delete
                    self.tau -= 1
                    self.tau_loc[c] = self.tau_loc.get(c, 0) - 1
                    if self.tau_loc[c] <= 0:
                        del self.tau_loc[c]
                    self.tau_loc[edge[0]] = self.tau_loc.get(edge[0], 0) - 1
                    if self.tau_loc[edge[0]] <= 0:
                        del self.tau_loc[edge[0]]
                    self.tau_loc[edge[1]] = self.tau_loc.get(edge[1], 0) - 1
                    if self.tau_loc[edge[1]] <= 0:
                        del self.tau_loc[edge[1]]

In [28]:
# EXPLOITING THE CLASS TRIEST-BASE

'''
At time _t
Graph G_t = (set of verteces V_t, set of edges E_t) 
'''
tb = TriestBase(4000)

print("Number of edges: ", len(data))
# simulate the stream of data
for  i in range(len(data)):
    tb.t += 1
    element = data[i]
    #print(edge)
    if tb.sampleEdge(tb.t):
        tb.S.append(element)
        tb.updateCounters('insert', element)
        
eps = max(1, (tb.t*(tb.t-1)*(tb.t-2))/(tb.M*(tb.M-1)*(tb.M-2)))
est_triangles = eps*tb.tau
print("Eps: ", eps)
print("Estimated triangles: ", est_triangles)
print(tb.tau)

Number of edges:  88234
Eps:  10740.85743886422
Estimated triangles:  3662632.3866526987
341


In [25]:
# Graphs and statistics 
import matplotlib.pyplot as plt
from collections import Counter
'''
cnt = Counter(C_1.values())
plt.bar(cnt.keys(), cnt.values())

plt.xlabel('Frequency')
plt.ylabel('Number of documents with that frequency')
plt.show()
'''

"\ncnt = Counter(C_1.values())\nplt.bar(cnt.keys(), cnt.values())\n\nplt.xlabel('Frequency')\nplt.ylabel('Number of documents with that frequency')\nplt.show()\n"